📒 [NB 1-2] 환자 코호트(Cohort) 추출 및 필터링
1. 실습 데이터 준비
조금 더 규모가 있는 가상의 EMR 데이터를 생성한다. 결측치(NaN)도 의도적으로 포함시킨다.

In [ ]:
import pandas as pd
import numpy as np

# 10명의 환자 데이터 생성
data = {
    'Patient_ID': [f'P{i:03d}' for i in range(1, 11)],
    'Age': [25, 65, 72, 34, 88, 55, 41, 62, 77, 29],
    'Gender': ['M', 'F', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'F'],
    'Diagnosis_Code': ['E11', 'I10', 'I10', 'J00', 'I10', 'E11', 'Normal', 'E11', 'C34', 'Normal'],
    # E11: 당뇨, I10: 고혈압, J00: 감기, C34: 폐암
    'Systolic_BP': [120, 145, 160, 110, 150, 130, 118, 135, np.nan, 115] # 혈압, 결측치 포함
}

df = pd.DataFrame(data)
print("원본 데이터:")
display(df)

원본 데이터:


,Patient_ID,Age,Gender,Diagnosis_Code,Systolic_BP
0,P001,25,M,E11,120.0
1,P002,65,F,I10,145.0
2,P003,72,M,I10,160.0
3,P004,34,F,J00,110.0
4,P005,88,M,I10,150.0
5,P006,55,F,E11,130.0
6,P007,41,M,Normal,118.0
7,P008,62,M,E11,135.0
8,P009,77,F,C34,NaN
9,P010,29,F,Normal,115.0


2. 조건에 맞는 환자 추출 (Boolean Indexing)
임상 연구에서 가장 중요한 '대상자 선정(Inclusion Criteria)' 과정을 실습한다.

In [2]:
# 시나리오 1: "65세 이상(Old)"인 노인 환자만 추출한다.
senior_cohort = df[df['Age'] >= 65]
print(f"\n[결과 1] 65세 이상 환자 수: {len(senior_cohort)}명")
display(senior_cohort)

# 시나리오 2: "고혈압(Code: I10)" 진단을 받은 환자만 추출한다.
htn_cohort = df[df['Diagnosis_Code'] == 'I10']
print(f"\n[결과 2] 고혈압 환자 수: {len(htn_cohort)}명")
display(htn_cohort)


[결과 1] 65세 이상 환자 수: 4명


,Patient_ID,Age,Gender,Diagnosis_Code,Systolic_BP
1,P002,65,F,I10,145.0
2,P003,72,M,I10,160.0
4,P005,88,M,I10,150.0
8,P009,77,F,C34,NaN



[결과 2] 고혈압 환자 수: 3명


,Patient_ID,Age,Gender,Diagnosis_Code,Systolic_BP
1,P002,65,F,I10,145.0
2,P003,72,M,I10,160.0
4,P005,88,M,I10,150.0


3. 복합 조건 필터링 (AND, OR)
여러 조건을 동시에 만족해야 하는 경우 연산자를 사용한다.
& (AND): 두 조건 모두 만족
| (OR): 둘 중 하나라도 만족
~ (NOT): 조건의 반대

In [ ]:
# 시나리오 3: "60세 이상" 이면서(AND) "고혈압(I10)"인 고위험군 환자 추출
high_risk_cohort = df[ (df['Age'] >= 60) & (df['Diagnosis_Code'] == 'I10') ]

print("\n[결과 3] 60세 이상 & 고혈압 환자 (고위험군):")
display(high_risk_cohort[['Patient_ID', 'Age', 'Diagnosis_Code', 'Systolic_BP']])

# 시나리오 4: 특정 질환군(당뇨 E11 또는 고혈압 I10)에 속하는 만성질환자 추출 (isin 활용)
chronic_diseases = ['E11', 'I10']
chronic_cohort = df[df['Diagnosis_Code'].isin(chronic_diseases)]

print(f"\n[결과 4] 만성질환(당뇨/고혈압) 환자 수: {len(chronic_cohort)}명")


[결과 3] 60세 이상 & 고혈압 환자 (고위험군):


,Patient_ID,Age,Diagnosis_Code,Systolic_BP
1,P002,65,I10,145.0
2,P003,72,I10,160.0
4,P005,88,I10,150.0



[결과 4] 만성질환(당뇨/고혈압) 환자 수: 6명


4. 결측치 필터링
데이터 품질 관리를 위해 중요 데이터가 비어있는 환자를 제외한다.

In [6]:
# 시나리오 5: 수축기 혈압(Systolic_BP) 기록이 없는 환자 확인 및 제외
missing_bp = df[df['Systolic_BP'].isnull()]
print("\n[확인] 혈압 기록이 없는 환자:")
display(missing_bp)

# 혈압 기록이 있는 환자만 남긴다 (dropna 대신 조건문 사용 예시)
clean_df = df[df['Systolic_BP'].notnull()]
print(f"\n[결과 5] 혈압 기록 유효 환자 데이터: {len(clean_df)}명")


[확인] 혈압 기록이 없는 환자:


,Patient_ID,Age,Gender,Diagnosis_Code,Systolic_BP
8,P009,77,F,C34,NaN



[결과 5] 혈압 기록 유효 환자 데이터: 9명
